# How to use

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Init Pyspark

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [3]:
spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.executor.memory", "6g")
    .config("spark.driver.memory", "6g")
    .getOrCreate()
)

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/19 17:14:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Import the data

In [4]:
from examples.make_data import make_transactions

In [5]:
make_transactions().head()

,transaction_id,consumer_id,paid_value,discount,product_type,buy_type,paymnent_date
0,0,29,1021.731939,10,h,f_4,2022-01-01 00:00:00.000000000
1,1,98,978.323923,10,c,f_7,2022-01-01 00:52:33.915391539
2,2,98,1017.947599,0,d,f_8,2022-01-01 01:45:07.830783078
3,3,38,1175.278194,25,a,f_2,2022-01-01 02:37:41.746174617
4,4,15,853.543907,10,a,f_1,2022-01-01 03:30:15.661566156


## Generate public

We will select the monthly active users as our public.

In [6]:
transactions = spark.createDataFrame(make_transactions())

public = transactions.select(F.col("consumer_id").alias("consumer_id_ref"), F.to_date(F.date_trunc("month", "paymnent_date")).alias("date_ref")).distinct()

## Generate Dataset

In [7]:
from autofeat.types import Dataset

In [8]:
df = Dataset(
    table=transactions,
    primary_key_col="transaction_id",
    table_join_key_col="consumer_id",
    table_join_date_col="paymnent_date",
    numerical_cols=["paid_value", "discount"],
    categorical_cols=["product_type", "buy_type"],
    public=public,
    public_join_key_col="consumer_id_ref",
    public_join_date_col="date_ref",
)


In [9]:
from autofeat import make_features

## numerical_statistics example

In [10]:
features = make_features.run(
    df=df,
    suites=[
        "numerical_statistics",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'sum___paid_value', 'sum___discount', 'mean___paid_value', 'mean___discount', 'stddev___paid_value', 'stddev___discount', 'min___paid_value', 'min___discount', 'max___paid_value', 'max___discount', 'kurtosis___paid_value', 'kurtosis___discount', 'skewness___paid_value', 'skewness___discount']
Number of features: 16


In [11]:
features.limit(5)

23/02/19 17:15:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


consumer_id_ref,date_ref,sum___paid_value,sum___discount,mean___paid_value,mean___discount,stddev___paid_value,stddev___discount,min___paid_value,min___discount,max___paid_value,max___discount,kurtosis___paid_value,kurtosis___discount,skewness___paid_value,skewness___discount
29,2022-01-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null
29,2022-02-01,12884.503346711668,205,991.1156420547437,15.76923076923077,102.66345300595485,10.963225241337865,831.0771002092232,0,1213.2540898678285,25,-0.09162914408204115,-1.5023466666666672,0.5059625417668084,-0.44986279389757616
26,2022-02-01,7508.382270897559,145,938.5477838621948,18.125,122.70809047827476,9.977653603356424,699.015702424566,0,1117.5160783905533,25,0.17455766080009605,-0.8945685616038936,-0.5859000078847572,-0.8306056504415191
26,2022-01-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null
29,2022-03-01,21210.539323911595,285,1010.0256820910283,13.571428571428571,103.5649985708696,10.856202966836188,831.0771002092232,0,1213.2540898678285,25,-0.42409637366403885,-1.6466942148760342,0.47735984043558566,-0.07354907590508099


## numerical_in_categorical_groups example

In [12]:
features = make_features.run(
    df=df,
    suites=[
        "numerical_in_categorical_groups",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'product_type=g__sum___paid_value', 'product_type=g__sum___discount', 'product_type=g__mean___paid_value', 'product_type=g__mean___discount', 'product_type=g__stddev___paid_value', 'product_type=g__stddev___discount', 'product_type=g__min___paid_value', 'product_type=g__min___discount', 'product_type=g__max___paid_value', 'product_type=g__max___discount', 'product_type=g__kurtosis___paid_value', 'product_type=g__kurtosis___discount', 'product_type=g__skewness___paid_value', 'product_type=g__skewness___discount', 'product_type=None__sum___paid_value', 'product_type=None__sum___discount', 'product_type=None__mean___paid_value', 'product_type=None__mean___discount', 'product_type=None__stddev___paid_value', 'product_type=None__stddev___discount', 'product_type=None__min___paid_value', 'product_type=None__min___discount', 'product_type=None__max___paid_value', 'product_type=None__max___discount', 'product_type=None__kurtosis___paid_value', 

In [13]:
features.limit(5)

23/02/19 17:15:29 WARN DAGScheduler: Broadcasting large task binary with size 1279.3 KiB


23/02/19 17:15:35 WARN DAGScheduler: Broadcasting large task binary with size 1279.3 KiB


consumer_id_ref,date_ref,product_type=g__sum___paid_value,product_type=g__sum___discount,product_type=g__mean___paid_value,product_type=g__mean___discount,product_type=g__stddev___paid_value,product_type=g__stddev___discount,product_type=g__min___paid_value,product_type=g__min___discount,product_type=g__max___paid_value,product_type=g__max___discount,product_type=g__kurtosis___paid_value,product_type=g__kurtosis___discount,product_type=g__skewness___paid_value,product_type=g__skewness___discount,product_type=None__sum___paid_value,product_type=None__sum___discount,product_type=None__mean___paid_value,product_type=None__mean___discount,product_type=None__stddev___paid_value,product_type=None__stddev___discount,product_type=None__min___paid_value,product_type=None__min___discount,product_type=None__max___paid_value,product_type=None__max___discount,product_type=None__kurtosis___paid_value,product_type=None__kurtosis___discount,product_type=None__skewness___paid_value,product_type=None__skewness___discount,product_type=f__sum___paid_value,product_type=f__sum___discount,product_type=f__mean___paid_value,product_type=f__mean___discount,product_type=f__stddev___paid_value,product_type=f__stddev___discount,product_type=f__min___paid_value,product_type=f__min___discount,product_type=f__max___paid_value,product_type=f__max___discount,product_type=f__kurtosis___paid_value,product_type=f__kurtosis___discount,product_type=f__skewness___paid_value,product_type=f__skewness___discount,product_type=e__sum___paid_value,product_type=e__sum___discount,product_type=e__mean___paid_value,product_type=e__mean___discount,product_type=e__stddev___paid_value,product_type=e__stddev___discount,product_type=e__min___paid_value,product_type=e__min___discount,product_type=e__max___paid_value,product_type=e__max___discount,product_type=e__kurtosis___paid_value,product_type=e__kurtosis___discount,product_type=e__skewness___paid_value,product_type=e__skewness___discount,product_type=h__sum___paid_value,product_type=h__sum___discount,product_type=h__mean___paid_value,product_type=h__mean___discount,product_type=h__stddev___paid_value,product_type=h__stddev___discount,product_type=h__min___paid_value,product_type=h__min___discount,product_type=h__max___paid_value,product_type=h__max___discount,product_type=h__kurtosis___paid_value,product_type=h__kurtosis___discount,product_type=h__skewness___paid_value,product_type=h__skewness___discount,product_type=d__sum___paid_value,product_type=d__sum___discount,product_type=d__mean___paid_value,product_type=d__mean___discount,product_type=d__stddev___paid_value,product_type=d__stddev___discount,product_type=d__min___paid_value,product_type=d__min___discount,product_type=d__max___paid_value,product_type=d__max___discount,product_type=d__kurtosis___paid_value,product_type=d__kurtosis___discount,product_type=d__skewness___paid_value,product_type=d__skewness___discount,product_type=c__sum___paid_value,product_type=c__sum___discount,product_type=c__mean___paid_value,product_type=c__mean___discount,product_type=c__stddev___paid_value,product_type=c__stddev___discount,product_type=c__min___paid_value,product_type=c__min___discount,product_type=c__max___paid_value,product_type=c__max___discount,product_type=c__kurtosis___paid_value,product_type=c__kurtosis___discount,product_type=c__skewness___paid_value,product_type=c__skewness___discount,product_type=b__sum___paid_value,product_type=b__sum___discount,product_type=b__mean___paid_value,product_type=b__mean___discount,product_type=b__stddev___paid_value,product_type=b__stddev___discount,product_type=b__min___paid_value,product_type=b__min___discount,product_type=b__max___paid_value,product_type=b__max___discount,product_type=b__kurtosis___paid_value,product_type=b__kurtosis___discount,product_type=b__skewness___paid_value,product_type=b__skewness___discount,product_type=a__sum___paid_value,product_type=a__sum___discount,product_type=a__mean___paid_value,product_type=a__mean___disc

## correlation example

In [14]:
features = make_features.run(
    df=df,
    suites=[
        "correlation",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'corr_between___paid_value_discount']
Number of features: 3


In [15]:
features.limit(5)

consumer_id_ref,date_ref,corr_between___paid_value_discount
29,2022-01-01,null
29,2022-02-01,-0.30725215225993513
26,2022-02-01,0.037880942524043705
26,2022-01-01,null
29,2022-03-01,-0.2310407779936052


In [16]:
features = make_features.run(
    df=df,
    suites=[
        "categorical_statistics",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'product_type=g__count___product_type', 'product_type=None__count___product_type', 'product_type=f__count___product_type', 'product_type=e__count___product_type', 'product_type=h__count___product_type', 'product_type=d__count___product_type', 'product_type=c__count___product_type', 'product_type=b__count___product_type', 'product_type=a__count___product_type', 'buy_type=f_3__count___buy_type', 'buy_type=f_5__count___buy_type', 'buy_type=None__count___buy_type', 'buy_type=f_8__count___buy_type', 'buy_type=f_4__count___buy_type', 'buy_type=f_2__count___buy_type', 'buy_type=f_9__count___buy_type', 'buy_type=f_1__count___buy_type', 'buy_type=f_7__count___buy_type', 'buy_type=f_6__count___buy_type', 'count___product_type', 'countDistinct___product_type', 'count___buy_type', 'countDistinct___buy_type']
Number of features: 25


In [17]:
features.limit(5)

consumer_id_ref,date_ref,product_type=g__count___product_type,product_type=None__count___product_type,product_type=f__count___product_type,product_type=e__count___product_type,product_type=h__count___product_type,product_type=d__count___product_type,product_type=c__count___product_type,product_type=b__count___product_type,product_type=a__count___product_type,buy_type=f_3__count___buy_type,buy_type=f_5__count___buy_type,buy_type=None__count___buy_type,buy_type=f_8__count___buy_type,buy_type=f_4__count___buy_type,buy_type=f_2__count___buy_type,buy_type=f_9__count___buy_type,buy_type=f_1__count___buy_type,buy_type=f_7__count___buy_type,buy_type=f_6__count___buy_type,count___product_type,countDistinct___product_type,count___buy_type,countDistinct___buy_type
70,2022-02-01,2,0,3,0,1,2,1,2,4,3,0,0,1,5,1,3,1,1,0,15,7,15,7
46,2022-03-01,1,0,1,0,1,1,5,1,1,0,1,0,2,1,1,2,2,1,1,11,7,11,8
55,2022-03-01,2,0,3,1,2,0,3,2,1,0,1,0,1,1,1,4,3,1,2,14,7,14,8
80,2022-11-01,2,0,1,3,0,1,2,3,4,3,1,0,3,3,0,2,1,1,2,16,7,16,8
1,2022-04-01,1,0,1,4,1,0,2,1,6,3,3,0,0,1,2,1,1,1,4,16,7,16,8


## first_observation_features example

In [18]:
features = make_features.run(
    df=df,
    suites=[
        "first_observation_features",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'first___paid_value', 'first___discount']
Number of features: 4


In [19]:
features.limit(5)

consumer_id_ref,date_ref,first___paid_value,first___discount
0,2022-01-01,null,null
0,2022-02-01,724.6474816548626,0
0,2022-03-01,724.6474816548626,0
0,2022-04-01,724.6474816548626,0
0,2022-05-01,989.642360404834,0


## last_observation_features example

In [20]:
features = make_features.run(
    df=df,
    suites=[
        "last_observation_features",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'last___paid_value', 'last___discount']
Number of features: 4


In [21]:
features.limit(5)

consumer_id_ref,date_ref,last___paid_value,last___discount
0,2022-01-01,null,null
0,2022-02-01,989.642360404834,0
0,2022-03-01,1068.3206046873418,25
0,2022-04-01,940.2798461945033,25
0,2022-05-01,1047.8739332006205,10


## lags example

In [22]:
features = make_features.run(
    df=df,
    suites=[
        "numerical_statistics",
        "lags",
    ],
    options={"n_lags": [1, 3, 4, 5]},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'sum___paid_value', 'sum___discount', 'mean___paid_value', 'mean___discount', 'stddev___paid_value', 'stddev___discount', 'min___paid_value', 'min___discount', 'max___paid_value', 'max___discount', 'kurtosis___paid_value', 'kurtosis___discount', 'skewness___paid_value', 'skewness___discount', 'lag=1_sum___paid_value', 'lag=1_sum___discount', 'lag=1_mean___paid_value', 'lag=1_mean___discount', 'lag=1_stddev___paid_value', 'lag=1_stddev___discount', 'lag=1_min___paid_value', 'lag=1_min___discount', 'lag=1_max___paid_value', 'lag=1_max___discount', 'lag=1_kurtosis___paid_value', 'lag=1_kurtosis___discount', 'lag=1_skewness___paid_value', 'lag=1_skewness___discount', 'lag=3_sum___paid_value', 'lag=3_sum___discount', 'lag=3_mean___paid_value', 'lag=3_mean___discount', 'lag=3_stddev___paid_value', 'lag=3_stddev___discount', 'lag=3_min___paid_value', 'lag=3_min___discount', 'lag=3_max___paid_value', 'lag=3_max___discount', 'lag=3_kurtosis___pa

In [23]:
features.limit(5)

consumer_id_ref,date_ref,sum___paid_value,sum___discount,mean___paid_value,mean___discount,stddev___paid_value,stddev___discount,min___paid_value,min___discount,max___paid_value,max___discount,kurtosis___paid_value,kurtosis___discount,skewness___paid_value,skewness___discount,lag=1_sum___paid_value,lag=1_sum___discount,lag=1_mean___paid_value,lag=1_mean___discount,lag=1_stddev___paid_value,lag=1_stddev___discount,lag=1_min___paid_value,lag=1_min___discount,lag=1_max___paid_value,lag=1_max___discount,lag=1_kurtosis___paid_value,lag=1_kurtosis___discount,lag=1_skewness___paid_value,lag=1_skewness___discount,lag=3_sum___paid_value,lag=3_sum___discount,lag=3_mean___paid_value,lag=3_mean___discount,lag=3_stddev___paid_value,lag=3_stddev___discount,lag=3_min___paid_value,lag=3_min___discount,lag=3_max___paid_value,lag=3_max___discount,lag=3_kurtosis___paid_value,lag=3_kurtosis___discount,lag=3_skewness___paid_value,lag=3_skewness___discount,lag=4_sum___paid_value,lag=4_sum___discount,lag=4_mean___paid_value,lag=4_mean___discount,lag=4_stddev___paid_value,lag=4_stddev___discount,lag=4_min___paid_value,lag=4_min___discount,lag=4_max___paid_value,lag=4_max___discount,lag=4_kurtosis___paid_value,lag=4_kurtosis___discount,lag=4_skewness___paid_value,lag=4_skewness___discount,lag=5_sum___paid_value,lag=5_sum___discount,lag=5_mean___paid_value,lag=5_mean___discount,lag=5_stddev___paid_value,lag=5_stddev___discount,lag=5_min___paid_value,lag=5_min___discount,lag=5_max___paid_value,lag=5_max___discount,lag=5_kurtosis___paid_value,lag=5_kurtosis___discount,lag=5_skewness___paid_value,lag=5_skewness___discount
0,2022-01-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,2022-02-01,8002.543578036849,55,1000.3179472546061,6.875,118.5476910585547,8.838834764831844,724.6474816548626,0,1092.4934907204822,25,1.8855027332009806,0.1335510204081629,-1.764323227323885,1.0496613364713396,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,2022-03-01,18183.864504218556,180,1010.2146946788087,10.0,88.82691760886172,11.504474832710557,724.6474816548626,0,1114.2036899159368,25,4.00337819774453,-1.6,-1.8871738944593603,0.447213595499958,8002.543578036849,55,1000.3179472546061,6.875,118.5476910585547,8.838834764831844,724.6474816548626,0,1092.4934907204822,25,1.8855027332009806,0.1335510204081629,-1.764323227323885,1.0496613364713396,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,2022-04-01,23073.719219865256,250,1003.2051834724024,10.869565217391305,91.68364350589783,11.246431142310668,724.6474816548626,0,1166.5396308621657,25,1.930602398031068,-1.6379455566406262,-1.00759236996,0.3177718469055919,18183.864504218556,180,1010.2146946788087,10.0,88.82691760886172,11.504474832710557,724.6474816548626,0,1114.2036899159368,25,4.00337819774453,-1.6,-1.8871738944593603,0.447213595499958,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,2022-05-01,27898.085780020647,335,996.3602064293088,11.964285714285714,89.04801902004334,11.331640863356228,783.539838218848,0,1169.5751292317634,25,0.015765997211552918,-1.7348126758682978,-0.04146256322646...,0.14211194159975876,23073.719219865256,250,1003.2051834724024,10.869565217391305

## increase_rate example

In [24]:
features = make_features.run(
    df=df,
    suites=[
        "numerical_statistics",
        "increase_rate",
    ],
    options={},
)

print(f"Features created: {features.columns}")

print(f"Number of features: {len(features.columns)}")

Features created: ['consumer_id_ref', 'date_ref', 'sum___paid_value', 'sum___discount', 'mean___paid_value', 'mean___discount', 'stddev___paid_value', 'stddev___discount', 'min___paid_value', 'min___discount', 'max___paid_value', 'max___discount', 'kurtosis___paid_value', 'kurtosis___discount', 'skewness___paid_value', 'skewness___discount', 'increase_rate_sum___paid_value', 'increase_rate_sum___discount', 'increase_rate_mean___paid_value', 'increase_rate_mean___discount', 'increase_rate_stddev___paid_value', 'increase_rate_stddev___discount', 'increase_rate_min___paid_value', 'increase_rate_min___discount', 'increase_rate_max___paid_value', 'increase_rate_max___discount', 'increase_rate_kurtosis___paid_value', 'increase_rate_kurtosis___discount', 'increase_rate_skewness___paid_value', 'increase_rate_skewness___discount']
Number of features: 30


In [25]:
features.limit(5)

consumer_id_ref,date_ref,sum___paid_value,sum___discount,mean___paid_value,mean___discount,stddev___paid_value,stddev___discount,min___paid_value,min___discount,max___paid_value,max___discount,kurtosis___paid_value,kurtosis___discount,skewness___paid_value,skewness___discount,increase_rate_sum___paid_value,increase_rate_sum___discount,increase_rate_mean___paid_value,increase_rate_mean___discount,increase_rate_stddev___paid_value,increase_rate_stddev___discount,increase_rate_min___paid_value,increase_rate_min___discount,increase_rate_max___paid_value,increase_rate_max___discount,increase_rate_kurtosis___paid_value,increase_rate_kurtosis___discount,increase_rate_skewness___paid_value,increase_rate_skewness___discount
0,2022-01-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,2022-02-01,8002.543578036849,55,1000.3179472546061,6.875,118.5476910585547,8.838834764831844,724.6474816548626,0,1092.4934907204822,25,1.8855027332009806,0.1335510204081629,-1.764323227323885,1.0496613364713396,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,2022-03-01,18183.864504218556,180,1010.2146946788087,10.0,88.82691760886172,11.504474832710557,724.6474816548626,0,1114.2036899159368,25,4.00337819774453,-1.6,-1.8871738944593603,0.447213595499958,1.2722606040065259,2.272727272727273,0.009893601780678211,0.45454545454545453,-0.2507073160540334,0.301582746911937,0.0,null,0.019872154278134026,0.0,1.123241789709885,-12.980440097799542,0.06963047656625515,-0.5739448715874762
0,2022-04-01,23073.719219865256,250,1003.2051834724024,10.869565217391305,91.68364350589783,11.246431142310668,724.6474816548626,0,1166.5396308621657,25,1.930602398031068,-1.6379455566406262,-1.00759236996,0.3177718469055919,0.2689117439536728,0.3888888888888889,-0.00693863516669...,0.08695652173913046,0.03216058796068272,-0.02242985396136...,0.0,null,0.04697160978723512,0.0,-0.5177566788172165,0.023715972900391347,-0.466083982552834,-0.2894405489834404
0,2022-05-01,27898.085780020647,335,996.3602064293088,11.964285714285714,89.04801902004334,11.331640863356228,783.539838218848,0,1169.5751292317634,25,0.015765997211552918,-1.7348126758682978,-0.04146256322646...,0.14211194159975876,0.20908491232752222,0.34,-0.00682310773096...,0.10071428571428562,-0.02874694313042...,0.00757660096499315,0.08127035290248173,null,0.002602139086654274,0.0,-0.9918336384396746,0.05913940108384488,-0.9588498638312356,-0.552786242759953
